In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random

import time

kafka_topic_name = "Topic1"
kafka_bootstrap_servers = 'localhost:9092'

spark = SparkSession \
        .builder \
        .appName("Structured Streaming ") \
        .master("local[*]") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [2]:
flower_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
        .option("subscribe", kafka_topic_name) \
        .option("startingOffsets", "earliest") \
        .load()

In [3]:
flower_df1 = flower_df.selectExpr("CAST(value AS STRING)", "timestamp")


flower_schema_string = "order_id INT,sepal_length DOUBLE,sepal_length DOUBLE,sepal_length DOUBLE,sepal_length DOUBLE,species STRING"



flower_df2 = flower_df1 \
        .select(from_csv(col("value"), flower_schema_string) \
                .alias("flower"), "timestamp")


flower_df3 = flower_df2.select("flower.*", "timestamp")

    
flower_df3.createOrReplaceTempView("flower_find");
song_find_text = spark.sql("SELECT * FROM flower_find")
flower_agg_write_stream = song_find_text \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("testedTable") \
        .start()

flower_agg_write_stream.awaitTermination(1)

False

In [7]:
df = spark.sql("SELECT * FROM testedTable")
df.show()

+--------+------------+------------+------------+------------+-----------+--------------------+
|order_id|sepal_length|sepal_length|sepal_length|sepal_length|    species|           timestamp|
+--------+------------+------------+------------+------------+-----------+--------------------+
|       0|         5.1|         3.5|         1.4|         0.2|Iris-setosa|2024-06-05 12:49:...|
|       1|         4.9|         3.0|         1.4|         0.2|Iris-setosa|2024-06-05 12:49:...|
|       2|         4.7|         3.2|         1.3|         0.2|Iris-setosa|2024-06-05 12:49:...|
|       3|         4.6|         3.1|         1.5|         0.2|Iris-setosa|2024-06-05 12:49:...|
|       4|         5.0|         3.6|         1.4|         0.2|Iris-setosa|2024-06-05 12:49:...|
|       5|         5.4|         3.9|         1.7|         0.4|Iris-setosa|2024-06-05 12:49:...|
|       6|         4.6|         3.4|         1.4|         0.3|Iris-setosa|2024-06-05 12:49:...|
|       7|         5.0|         3.4|    

In [8]:
df.count()

52

In [9]:
df.count()

62

In [10]:
df.count()

72